In [0]:
dbutils.widgets.text("experiment", "/Workspace/9900-f18a-cake/classifier")
dbutils.widgets.text("child_notebook", "/Shared/mt-method2/worker_child_pycall")
dbutils.widgets.text("labels_table", "cb_prod.`comp9300-9900-f18a-cake`.node_labels")
dbutils.widgets.text("id_col", "biosample_id")
dbutils.widgets.text("label_col", "direct_label")

In [0]:
dbutils.widgets.text("repo_src", "/Workspace/9900-f18a-cake/mt-method2/src")
dbutils.widgets.text("rf_n_jobs", "1")
dbutils.widgets.text("cv_n_jobs", "1")
dbutils.widgets.text("prefilter_topk", "200")
dbutils.widgets.text("prefilter_scan_max", "5000")
dbutils.widgets.text("prefilter_chunk_size", "1000")
dbutils.widgets.dropdown("disable_dm", "true", ["true","false"])
dbutils.widgets.text("raise_on_error", "true")

In [0]:
dbutils.widgets.text("include_nodes_csv", "") 
dbutils.widgets.dropdown("launch_mode", "sequential", ["sequential","plan_only"])

In [0]:
EXPERIMENT     = dbutils.widgets.get("experiment")
CHILD_NOTEBOOK = dbutils.widgets.get("child_notebook")
LABELS_TABLE   = dbutils.widgets.get("labels_table")
ID_COL         = dbutils.widgets.get("id_col")
LABEL_COL      = dbutils.widgets.get("label_col")
INCLUDE_CSV    = dbutils.widgets.get("include_nodes_csv").strip()
LAUNCH_MODE    = dbutils.widgets.get("launch_mode")

In [0]:
REPO_SRC       = dbutils.widgets.get("repo_src")
RF_N_JOBS      = dbutils.widgets.get("rf_n_jobs")
CV_N_JOBS      = dbutils.widgets.get("cv_n_jobs")
PREF_TOPK      = dbutils.widgets.get("prefilter_topk")
PREF_SCAN_MAX  = dbutils.widgets.get("prefilter_scan_max")
PREF_CHUNK     = dbutils.widgets.get("prefilter_chunk_size")
DISABLE_DM     = dbutils.widgets.get("disable_dm")
RAISE_ON_ERROR = dbutils.widgets.get("raise_on_error")

In [0]:
print("Experiment:", EXPERIMENT)
print("Child:", CHILD_NOTEBOOK)
print("Labels table:", LABELS_TABLE)
print("Mode:", LAUNCH_MODE)